consulta 1: Calcula la distancia promedio y el tiempo promedio de viaje en minutos para todos los viajes realizados en diciembre 2019

In [3]:
import pandas as pd
import os

file_path = os.path.join("..", "data", "processed", "tripdata_clean.csv")
df= pd.read_csv(file_path)

# Calcular el promedio de las columnas
distancia_promedio = df['trip_distance'].mean()
tiempo_promedio_viaje = df['trip_duration_min'].mean()

# Redondear resultados a 2 decimales
distancia_promedio = round(distancia_promedio, 2)
tiempo_promedio_viaje = round(tiempo_promedio_viaje, 2)

# Mostrar resultados
print(f"Distancia promedio en millas: {distancia_promedio}")
print(f"Tiempo promedio de viaje en minutos: {tiempo_promedio_viaje}")

Distancia promedio en millas: 2.98
Tiempo promedio de viaje en minutos: 15.07


Consulta 2: Determina el método de pago más común y cuántos viajes lo usaron. 

In [4]:
import pandas as pd
import os

file_path = os.path.join("..", "data", "processed", "tripdata_clean.csv")
df = pd.read_csv(file_path)

# Diccionario que traduce valores numéricos a nombres de métodos de pago
metodos_pago_diccionario = {
    1: 'tarjeta de crédito',
    2: 'efectivo',
    3: 'sin cargo',
    4: 'en disputa',
    5: 'desconocido',
    6: 'viaje cancelado' 
}

# Mapea los nombres de método de pago en la columna original y luego contar sus ocurrencias.
conteo = df['payment_type'].map(metodos_pago_diccionario).value_counts()

# Obtener método más común y cantidad de viajes que lo usaron
metodo_mas_comun = conteo.idxmax()
cantidad_viajes = conteo.max()

print(f"Método de pago más común: {metodo_mas_comun}")
print(f"Cantidad de viajes que usaron ese método: {cantidad_viajes}")

Método de pago más común: tarjeta de crédito
Cantidad de viajes que usaron ese método: 4736318


Consulta 3: Calcula la propina promedio según el tipo de pago pero sólo para viajes donde la propina fue mayor que 0. 

In [5]:
import pandas as pd
import os

file_path = os.path.join("..", "data", "processed", "tripdata_clean.csv")
df = pd.read_csv(file_path)

# Diccionario que traduce valores numéricos a nombres de métodos de pago
metodos_pago_dict = {
    1: 'tarjeta de crédito',
    2: 'efectivo',
    3: 'sin cargo',
    4: 'en disputa',
    5: 'desconocido',
    6: 'viaje cancelado' 
}

# 1. se filtran los viajes con propina mayor a 0 y se calcula el promedio de propina por tipo de pago
promedio_propina = (
    df.loc[df['tip_amount'] > 0]
      .groupby('payment_type')['tip_amount']
      .mean()
)

# 2. se traducen los metodos de pago usando el diccionario
promedio_propina = promedio_propina.rename(index=metodos_pago_dict)

# 3. para un mejor orden se resetea el índice y se renombran las columnas 
# ademas de redondear a 2 decimales
df_resultado = (
    promedio_propina.round(2)
    .reset_index()
    .rename(columns={'payment_type': 'Método_de_pago', 'tip_amount': 'Propina_promedio'})
)

print(df_resultado)

       Método_de_pago  Propina_promedio
0  tarjeta de crédito              3.25
1            efectivo              2.90
2           sin cargo              3.48
3          en disputa              2.68


consulta 4: Muestra el porcentaje de viajes correspondientes a cada tipo de tarifa, ordenado de mayor a menor participación. 

In [6]:
import pandas as pd
import os

# Carga de datos (como la tienes definida)
file_path = os.path.join("..", "data", "processed", "tripdata_clean.csv")
df = pd.read_csv(file_path)

# 1. diccionario para mapear los códigos de tarifa a nombres legibles
diccionario_tarifas = {
    1: 'Tarifa estandar',
    2: 'JFK',
    3: 'Newark',    
    4: 'Nassau o Westchester',
    5: 'Tarifa negociada',
    6: 'Tarifa grupal',
    99: 'Desconocido'
}

# 2. contar la frecuencia de cada tipo de tarifa
conteo_tarifas = df['ratecodeid'].value_counts()

# 3. Cálculo del porcentaje de participación
total_viajes = conteo_tarifas.sum()
porcentaje_participación = (conteo_tarifas / total_viajes) * 100

# 4. para facilitar la visualización
# crear un DataFrame con los resultados y redondear el porcentaje a 2 decimales
df_resultado = pd.DataFrame({
    'Total_Viajes': conteo_tarifas,
    'Porcentaje_participación': porcentaje_participación.round(2)
})

# 5. Aplicar el mapeo para mostrar el nombre legible de la tarifa
df_resultado = df_resultado.rename(index=diccionario_tarifas)

# 6. Ordenar por porcentaje de mayor a menor participación y renombrar el índice
df_resultado = df_resultado.sort_values(by='Porcentaje_participación', ascending=False)
df_resultado = df_resultado.rename_axis('Tipo de Tarifa')

print(df_resultado)

                      Total_Viajes  Porcentaje_participación
Tipo de Tarifa                                              
Tarifa estandar            6402706                     96.78
JFK                         171231                      2.59
Tarifa negociada             18950                      0.29
Newark                       17169                      0.26
Nassau o Westchester          5106                      0.08
Desconocido                    264                      0.00
Tarifa grupal                   12                      0.00


consulta 5: 
Clasifica los viajes según su distancia (Trip_distance) en tres categorías: 

Corto: menos de 2 millas 

Medio: entre 2 y 10 millas 

Largo: más de 10 millas 

Muestra cuántos viajes hay en cada categoría. 

In [7]:
import numpy as np
import os

file_path = os.path.join("..", "data", "processed", "tripdata_clean.csv")
df = pd.read_csv(file_path)

# 1. Definir los límites y las etiquetas de las categorías
limites_distancia = [0, 2.0, 10.0, float('inf')]
etiquetas_distancia = ['Corto', 'Medio', 'Largo']

# Definición de las condiciones
condiciones = [
    df['trip_distance'] < 2,                  # Corto: menos de 2 millas
    (df['trip_distance'] >= 2) & (df['trip_distance'] <= 10), # Medio: entre 2 y 10 millas
    df['trip_distance'] > 10                  # Largo: más de 10
]

# Crear la columna de categoría
df['Categoria_Distancia'] = np.select(
    condiciones,
    etiquetas_distancia,
    default='Desconocido' 
)

# 3. Contar cuántos viajes hay en cada categoría
conteo_categorias = df['Categoria_Distancia'].value_counts()

# 4. resetear el índice para una mejor presentación
df_conteo_categorias = conteo_categorias.reset_index()
df_conteo_categorias.columns = ['Categoría de Distancia', 'Número de Viajes']


print(df_conteo_categorias)

  Categoría de Distancia  Número de Viajes
0                  Corto           3926694
1                  Medio           2295375
2                  Largo            393369


creacion del df para exportar los resultados a la app 

In [10]:
import pandas as pd
import os

# Crear el DataFrame con los resultados consolidados 
data = {
    'ID_Consulta': [1, 1, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5],
    'Consulta': [
        'Distancia promedio', 'tiempo promedio de viaje en minutos',
        'Método de pago más común', 'Método de pago más común',
        'Propina promedio según el tipo de pago (propina > 0)', 'Propina promedio según el tipo de pago (propina > 0)',
        'Propina promedio según el tipo de pago (propina > 0)', 'Propina promedio según el tipo de pago (propina > 0)',
        'Porcentaje de viajes correspondientes a cada tipo de tarifa', 'Porcentaje de viajes correspondientes a cada tipo de tarifa',
        'Porcentaje de viajes correspondientes a cada tipo de tarifa', 'Porcentaje de viajes correspondientes a cada tipo de tarifa',
        'Porcentaje de viajes correspondientes a cada tipo de tarifa', 'Porcentaje de viajes correspondientes a cada tipo de tarifa',
        'Porcentaje de viajes correspondientes a cada tipo de tarifa',
        'Conteo de viajes por categoría de distancia', 'Conteo de viajes por categoría de distancia', 'Conteo de viajes por categoría de distancia'
    ],
    'Etiqueta': [
        'Distancia promedio', 'Tiempo promedio', 
        # Modificaciones para Consulta 2
        'Método de pago más usado', 'Cantidad de viajes que lo usaron',
        'tarjeta de crédito', 'efectivo', 'sin cargo', 'en disputa',
        'Tarifa estandar', 'JFK', 'Tarifa negociada', 'Newark', 'Nassau o Westchester', 'Desconocido', 'Tarifa grupal',
        'Corto', 'Medio', 'Largo'
    ],
    'Valor Numerico': [
        2.98, 15.07, 
        # Modificaciones para Consulta 2
        float('nan'), 4736318.00, # El valor numérico del Método de pago se establece como NaN
        # Resto de valores
        3.25, 2.90, 3.48, 2.68, 96.78, 2.59, 0.29, 0.26, 0.08, 0.00, 0.00, 3926694, 2295375, 393369
    ],
    'Unidad': [
        'millas', 'minutos', 
        # Modificaciones para Consulta 2
        'tarjeta de crédito', 'viajes', 
        # Resto de unidades
        '$', '$', '$', '$', '%', '%', '%', '%', '%', '%', '%', 'viajes', 'viajes', 'viajes'
    ]
}
df_resultados_consolidados = pd.DataFrame(data)


# 1. Definir la ruta del directorio
BASE_DIR = os.path.dirname(os.getcwd()) 
processed_dir = os.path.join(BASE_DIR, "data", "processed")

# 2. Asegurar que la carpeta existe (por si acaso)
os.makedirs(processed_dir, exist_ok=True)

# Ruta final del archivo CSV
output_path = os.path.join(processed_dir, "resultados_consolidados.csv")

# 4. Guardar el DataFrame en la ruta especificada
df_resultados_consolidados.to_csv(output_path, index=False)

# imprimir resultado
print(f"Archivo resultados_consolidados.csv creado con éxito en: {output_path}")

Archivo resultados_consolidados.csv creado con éxito en: c:\Users\vegas\OneDrive\Documentos\GitHub\nyc-taxi-demanda-2019\data\processed\resultados_consolidados.csv
